In [ ]:
import pandas as pd

pd.set_option("display.max_columns", None)
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np

import plotly.figure_factory as ff
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

%load_ext nb_black

In [ ]:
df = pd.read_csv("../data/train.csv", index_col=0)

In [ ]:
df.info()

## calculate assets

We can calculate assets using 2 different methods

1. logarithm of total assets (1 missing and 0 zero)
2. working capital and working capital / total assets (1 missing and 0 zero) 

The second method have 31 rows which have negative assets

In [ ]:
def calculate_assets(df):
    new_df = df.assign(
        **{
            "total_assets_method_1": 10 ** df["logarithm of total assets"],
            "total_assets_method_2": lambda df: df["working capital"]
            / df["working capital / total assets"],
        }
    )
    return new_df[
        [
            "logarithm of total assets",
            "working capital",
            "working capital / total assets",
            "total_assets_method_1",
            "total_assets_method_2",
            "bankruptcy_label",
        ]
    ]

In [ ]:
preprocessed_df = calculate_assets(df)


def check_negative_assets(df, asset_col):
    mask = df[asset_col] < 0
    print(f"Number of rows which have negative assets: {len(df[mask])}")
    return df[mask]


check_negative_assets(preprocessed_df, "total_assets_method_2")

## calculate sales 

We can calculate sales using 7 different methods

1. sales / total assets (0 missing and 21 zeros)
2. total sales / total assets (1 missing and 4 zeros)
3. gross profit / total assets and gross profit / sales (21 missing & 8 zeros) 
4. net profit / total assets and net profit / sales (21 missing & 45 zeros)
5. profit on sales / total assets and profit on sales / sales (21 missing & 51 zeros)
6. EBITDA / total assets and EBITDA / sales (21 missing & 213 zeros)
7. profit on operating activities  / total assets and profit on operating activities  / sales (21 missing & 892 zeros)

We will check the first four methods.

Sales calculated using Method 1 vs Method 3 have 3000 rows where absolute difference is more than 1000.

Proposed solution:
8. Combine first method 3, then method 1

In [ ]:
def calculate_sales(df):
    new_df = df.assign(
        **{
            "total_assets": 10 ** df["logarithm of total assets"],
            "sales_method_1": lambda df: df["sales / total assets"]
            * df["total_assets"],
            "sales_method_2": lambda df: df["total sales / total assets"]
            * df["total_assets"],
            "net_profit": lambda df: df["net profit / total assets"]
            * df["total_assets"],
            "gross_profit": lambda df: df["gross profit / total assets"]
            * df["total_assets"],
            "sales_method_3": lambda df: df["gross_profit"]
            / df["gross profit / sales"],
            "sales_method_4": lambda df: df["net_profit"] / df["net profit / sales"],
            "sales_method_8": lambda df: df["sales_method_3"].combine_first(
                df["sales_method_1"]
            ),
        }
    )
    return new_df[
        [
            "logarithm of total assets",
            "sales / total assets",
            "total_assets",
            "sales_method_1",
            "sales_method_2",
            "sales_method_3",
            "sales_method_4",
            "sales_method_8",
            "net_profit",
            "gross_profit",
            "bankruptcy_label",
        ]
    ]

In [ ]:
def compare_col(df, col1, col2, atol, sample=True):
    mask = np.isclose(df[col1], df[col2], atol=atol)
    print(
        f"Number of rows which have absolute difference more than {atol}: {len(df[~mask])}"
    )
    if sample:
        return df[~mask].sample(10)
    else:
        return df[~mask]


# calculate_sales(df).sample(20)
preprocessed_df = calculate_sales(df)
compare_col(preprocessed_df, "sales_method_3", "sales_method_8", atol=1000)

In [ ]:
preprocessed_df = calculate_sales(df)

sns.distplot(preprocessed_df["sales_method_1"], label="Sales Method 1")
sns.distplot(preprocessed_df["sales_method_2"], label="Sales Method 2")
sns.distplot(preprocessed_df["sales_method_3"], label="Sales Method 3")
plt.legend()

### assert net profit less than sales

- There are 32 cases where net profit is bigger than sales when using method 1
- There are 41 cases where net profit is bigger than sales when using method 3

In [ ]:
preprocessed_df = calculate_sales(df)
mask = preprocessed_df["net_profit"] < preprocessed_df["sales_method_3"]
print(len(preprocessed_df[~mask].dropna()))
preprocessed_df[~mask].dropna()

In [ ]:
df['sales / total assets'].describe()

In [ ]:
mask = df["sales / total assets"] < df["net profit / total assets"]
print(len(df[mask]))
df.loc[mask, ["sales / total assets", "net profit / total assets"]]

## check depreciation

Depreciation can be calculated via 3 different methods:

1. '(gross profit + depreciation) / sales' (21 missing and 8 zeros)
2. (gross profit + depreciation) / total liabilities' (19 missing and 1 zero)
3. '(net profit + depreciation) / total liabilities' (19 missing and 8 zeros)

Note: Total liabilities is first calculated via total liabilities / total assets (1 missing and 18 zeros)

If we use sales method 1 to reconcile the depreciation calculated using different methods, we get 2919 rows where the depreciation difference is more than 100.

However, if we use sales method 3 to reconcile the depreciation, we only get 42 rows where the depreciation difference is more than 100. Furthermore, the reason majority of these 42 rows cannot reconcile is because
either depreciation method 1 is null or depreciation method 2 is null.

Extra:
We can combine depreciation method 2 with depreciation method 1 to reduce the null values of depreciations

In [ ]:
def check_depreciation(df, sales_method):
    new_df = df.assign(
        **{
            "total_assets": 10 ** df["logarithm of total assets"],
            "sales_method_1": lambda df: df["sales / total assets"]
            * df["total_assets"],
            "sales_method_2": lambda df: df["total sales / total assets"]
            * df["total_assets"],
            "net_profit": lambda df: df["net profit / total assets"]
            * df["total_assets"],
            "gross_profit": lambda df: df["gross profit / total assets"]
            * df["total_assets"],
            "sales_method_3": lambda df: df["gross_profit"]
            / df["gross profit / sales"],
            "sales_method_4": lambda df: df["net_profit"] / df["net profit / sales"],
            "sales_method_8": lambda df: df["sales_method_3"].combine_first(
                df["sales_method_1"]
            ),
            "total_liabilities": lambda df: df["total liabilities / total assets"]
            * df["total_assets"],
            "depreciation_method_1": lambda df: (
                df["(gross profit + depreciation) / sales"] * df[sales_method]
            )
            - df["gross_profit"],
            "depreciation_method_2": lambda df: (
                df["(gross profit + depreciation) / total liabilities"]
                * df["total_liabilities"]
            )
            - df["gross_profit"],
            "depreciation_method_3": lambda df: (
                df["(net profit + depreciation) / total liabilities"]
                * df["total_liabilities"]
            )
            - df["net_profit"],
            "depreciation_method_4": lambda df: df[
                "depreciation_method_2"
            ].combine_first(df["depreciation_method_1"]),
        }
    )
    return new_df[
        [
            "logarithm of total assets",
            "sales / total assets",
            "total_assets",
            "sales_method_1",
            "sales_method_2",
            "sales_method_3",
            "sales_method_4",
            "sales_method_8",
            "depreciation_method_1",
            "depreciation_method_2",
            "depreciation_method_3",
            "depreciation_method_4",
            "bankruptcy_label",
        ]
    ]

In [ ]:
preprocessed_df = check_depreciation(df, sales_method="sales_method_3")

compare_col(
    preprocessed_df,
    "depreciation_method_1",
    "depreciation_method_2",
    atol=100,
    sample=True,
)

In [ ]:
preprocessed_df.isna().sum()